# Mozambique AA for Cholera – Threshold Analysis

This notebook supports the development of Anticipatory Action (AA) for cholera in Mozambique. It focuses on exploring and validating outbreak thresholds as defined in the national cholera preparedness and response plan.

The analysis leverages historical cholera case data and related risk indicators stored securely in Azure Blob Storage. Due to the sensitivity of the data, the data is not publicly available.

**Objectives:**
- Understand the thresholds set by the national plan and assess their applicability.
- Explore alternative data-driven thresholds to inform early action.

> **Note:** Ensure that access credentials for the Blob Storage container are configured before running the data loading section.


In [4]:
# Data and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Azure Blob Storage
from azure.storage.blob import ContainerClient, BlobClient
import io

# Display settings
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

In [8]:
DEV_BLOB_SAS = os.getenv("DSCI_AZ_BLOB_DEV_SAS")
DEV_BLOB_NAME = "imb0chd0dev"
DEV_BLOB_URL = f"https://{DEV_BLOB_NAME}.blob.core.windows.net/"
CONTAINER_NAME = "projects"
BLOB_PATH = "ds-aa-moz-cholera/data/"
container_url = f"{DEV_BLOB_URL}{CONTAINER_NAME}?{DEV_BLOB_SAS}"
container_client = ContainerClient.from_container_url(container_url)

In [12]:
excel_blobs = [blob.name for blob in container_client.list_blobs(name_starts_with=BLOB_PATH) if blob.name.endswith(".xlsx")]
dataframes = {}
for blob_name in excel_blobs:
    blob_url = f"{DEV_BLOB_URL}{CONTAINER_NAME}/{blob_name}?{DEV_BLOB_SAS}"
    blob_client = BlobClient.from_blob_url(blob_url)
    blob_data = blob_client.download_blob().readall()
    df = pd.read_excel(io.BytesIO(blob_data))
    key = os.path.basename(blob_name).replace(".xlsx", "")
    dataframes[key] = df

## Data Cleaning